In [ ]:
import zipfile
import os
from PIL import Image

In [ ]:
zip_path = '/content/archive.zip'
extract_path = '/content/extracted_dataset'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Successfully extracted {zip_path} to {extract_path}")

Successfully extracted /content/archive.zip to /content/extracted_dataset


In [ ]:
extracted_dataset_path = '/content/extracted_dataset/Lung X-Ray Image/Lung X-Ray Image'
categories = [d for d in os.listdir(extracted_dataset_path) if os.path.isdir(os.path.join(extracted_dataset_path, d))]
print("Categories:", categories)

Categories: ['Viral Pneumonia', 'Normal', 'Lung_Opacity']


In [ ]:
image_counts = {}
image_dimensions = {}

for category in categories:
    category_path = os.path.join(extracted_dataset_path, category)
    image_files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    image_counts[category] = len(image_files)

    sample_images = image_files[:5]
    dimensions = []
    for image_file in sample_images:
        image_path = os.path.join(category_path, image_file)
        try:
            with Image.open(image_path) as img:
                dimensions.append(img.size + (len(img.getbands()),))
        except Exception as e:
            print(f"Could not open image {image_path}: {e}")
    image_dimensions[category] = dimensions

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
extracted_dataset_path = '/content/extracted_dataset/Lung X-Ray Image/Lung X-Ray Image'


In [ ]:
IMG_WIDTH, IMG_HEIGHT = 299, 299
image_paths = []
labels = []

In [ ]:
for category in os.listdir(extracted_dataset_path):
    category_path = os.path.join(extracted_dataset_path, category)
    if os.path.isdir(category_path):
        for image_file in os.listdir(category_path):
            image_path = os.path.join(category_path, image_file)
            if os.path.isfile(image_path):
                image_paths.append(image_path)
                labels.append(category)

df = pd.DataFrame({'image_path': image_paths, 'label': labels})

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['label'])


In [ ]:
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)
print("\nTraining set label distribution:\n", train_df['label'].value_counts())
print("\nTesting set label distribution:\n", test_df['label'].value_counts())

Training set shape: (2432, 2)
Testing set shape: (1043, 2)

Training set label distribution:
 label
Normal             875
Lung_Opacity       787
Viral Pneumonia    770
Name: count, dtype: int64

Testing set label distribution:
 label
Normal             375
Lung_Opacity       338
Viral Pneumonia    330
Name: count, dtype: int64


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 32

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Train generator classes:", train_generator.class_indices)
print("Test generator classes:", test_generator.class_indices)

Found 2432 validated image filenames belonging to 3 classes.
Found 1043 validated image filenames belonging to 3 classes.
Train generator classes: {'Lung_Opacity': 0, 'Normal': 1, 'Viral Pneumonia': 2}
Test generator classes: {'Lung_Opacity': 0, 'Normal': 1, 'Viral Pneumonia': 2}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf


In [ ]:
num_classes = len(train_generator.class_indices)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))



In [ ]:
def f1_score(y_true, y_pred):
    true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', Precision(), Recall(), f1_score]) # Include accuracy, precision, recall, and custom f1_score

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 9, 9, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 41472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    10,617,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,332,547 (96.64 MB)

 Trainable params: 10,617,859 (40.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


76/76 ━━━━━━━━━━━━━━━━━━━━ 61s 478ms/step - accuracy: 0.6241 - f1_score: 0.6001 - loss: 2.6258 - precision: 0.6353 - recall: 0.5878 - val_accuracy: 0.8691 - val_f1_score: 0.8677 - val_loss: 0.3550 - val_precision: 0.8727 - val_recall: 0.8633
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 37s 491ms/step - accuracy: 0.9154 - f1_score: 0.9154 - loss: 0.2412 - precision: 0.9186 - recall: 0.9123 - val_accuracy: 0.8721 - val_f1_score: 0.8718 - val_loss: 0.3012 - val_precision: 0.8773 - val_recall: 0.8662
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 40s 524ms/step - accuracy: 0.9487 - f1_score: 0.9497 - loss: 0.1694 - precision: 0.9508 - recall: 0.9487 - val_accuracy: 0.8711 - val_f1_score: 0.8697 - val_loss: 0.3244 - val_precision: 0.8723 - val_recall: 0.8672
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 40s 510ms/step - accuracy: 0.9636 - f1_score: 0.9633 - loss: 0.1052 - precision: 0.9639 - recall: 0.9627 - val_accuracy: 0.8867 - val_f1_score: 0.8879 - val_loss: 0.2719 - val_precision: 0.8901 - val_recall: 0.8857

In [ ]:
loss, accuracy, precision, recall, f1 = model.evaluate(test_generator)

print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

33/33 ━━━━━━━━━━━━━━━━━━━━ 31s 946ms/step - accuracy: 0.8297 - f1_score: 0.8303 - loss: 0.4408 - precision_1: 0.8323 - recall_1: 0.8285

Test Loss: 0.4392
Test Accuracy: 0.8341
Test Precision: 0.8353
Test Recall: 0.8313
Test F1 Score: 0.8321


This notebook demonstrates the process of building and training a Convolutional Neural Network (CNN) for lung disease classification using a dataset downloaded from Kaggle.

1.  **Data Loading and Exploration**: The notebook starts by downloading the dataset from Kaggle using the `opendatasets` library. It then explores the dataset structure, identifies the class directories, and counts the number of images in each class.
2.  **Dataset Preparation**: The images are loaded into TensorFlow datasets, splitting the data into training and validation sets. The image size is set to 224x224 pixels, and the batch size is 32.
3.  **Model Building**: A sequential CNN model is built with several convolutional and max pooling layers to extract features from the images, followed by dense layers for classification.
4.  **Model Compilation and Training**: The model is compiled with the Adam optimizer and Sparse Categorical Crossentropy loss. Adam is a good default choice for an optimizer due to its efficiency, and Sparse Categorical Crossentropy is the appropriate loss function for multi-class classification with integer labels when the model outputs logits.
